In [2]:
from fastai.vision import * #import from vision to use the classification interpreter
from pathlib import Path
import pdb
import seaborn as sns
from sklearn.decomposition import PCA
from timeseries import TimeSeriesItem, TimeSeriesList, UCRArchive

In [3]:
ucr = UCRArchive()

In [4]:
class TSClassifier(torch.nn.Module):
    def __init__(self,seqLen,numClasses):
        super().__init__()
        self.conv = WideResNet1d(3,3,6)
        self.hidden= torch.nn.Linear(self.conv.nf,300)
        self.out = torch.nn.Linear(300,numClasses)
              
    def forward(self,ts):
        ts = self.conv(ts.unsqueeze(1))
        return self.out(torch.relu(self.hidden(ts)))
    
class TSAE(torch.nn.Module):
    def __init__(self,seqLen,latentDim=24):
        super().__init__()
        #self.conv = ConvBase(seqLen)
        self.conv = WideResNet1d(3,3,6)
        self.mean = torch.nn.Linear(self.conv.nf,latentDim)
        self.logvar = torch.nn.Linear(self.conv.nf,latentDim)
        

        layers = []
        for a,b in [(latentDim,100),(100,200),(200,300)]:
            layers += bn_drop_lin(a,b,actn=torch.nn.ReLU())
        self.lin = torch.nn.Sequential(*layers)
        self.out = torch.nn.Linear(300,seqLen)

    def forward(self,ts):
        seqLen = ts.shape[1]
        ts = self.conv(ts.unsqueeze(1)).squeeze(-1)
        #mean, logvar = self.mean(ts), self.logvar(ts)
        mean = self.mean(ts)
        
        ls = mean
#         if self.training:
#             std = torch.exp(0.5 * logvar)
#             eps = torch.randn_like(std)
#             ls = eps.mul(std).add_(mean)
        return self.out(self.lin(ls))          
        #return self.out(self.lin(ls)), mean, logvar

In [17]:
def EvaluateSeries(dataset_name):
    out = [dataset_name]
    print(f"Evaluating {dataset_name}")
    try:
        data = (TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)
                          .split_by_csv_name([f"{dataset_name}_TEST.tsv"])
                          .label_from_col())
        dslen = min(len(data.train),len(data.valid))
        bs = min(64,dslen//10)
        data = data.databunch(num_workers=0,bs=bs)

        dataAE = (TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)
                            .no_split()
                            .label_from_self()
                            .databunch(num_workers=0,bs=bs))

        learn = Learner(data,TSClassifier(len(data.train_ds[0][0].data),data.train_ds.c),loss_func=F.cross_entropy,metrics=[accuracy])
        learn.fit_one_cycle(10,1e-3,wd=0.2)
        learn.fit_one_cycle(10,1e-4,wd=0.2)
        out.append(max([m[0].item() for m in learn.recorder.metrics]))

        learnAE = Learner(dataAE,TSAE(len(data.train_ds[0][0].data)),loss_func=F.mse_loss)
        learnAE.fit_one_cycle(10,1e-2)
        learnAE.fit_one_cycle(10,1e-3)
        out.append(learnAE.validate(dataAE.train_dl)[0])
        
        learnDT = Learner(data,TSClassifier(len(data.train_ds[0][0].data),data.train_ds.c),loss_func=F.cross_entropy,metrics=[accuracy],
                 callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learnDT.split(split_model(learnDT.model,[*learnDT.model.conv.split_groups,learnDT.model.out]))
        learnDT.model.conv.load_state_dict(learnAE.model.conv.state_dict())
        learnDT.freeze_to(-1)
        learnDT.fit_one_cycle(10,1e-2)
        out.append(max([m[0].item() for m in learnDT.recorder.metrics]))
        learnDT.unfreeze()
        learnDT.fit_one_cycle(10,slice(1e-5,1e-2))
        out.append(max([m[0].item() for m in learnDT.recorder.metrics]))

    except:
        pass
    
    return out

In [ ]:
results = [EvaluateSeries(d) for d in ucr.list_datasets()]

epoch,train_loss,valid_loss,accuracy
1,nan,nan,0.100000
2,nan,nan,0.100000
3,nan,nan,0.100000
4,nan,nan,0.100000
5,nan,nan,0.100000
6,nan,nan,0.100000
7,nan,nan,0.100000
8,nan,nan,0.100000
9,nan,nan,0.100000
10,nan,nan,0.100000


epoch,train_loss,valid_loss,accuracy
1,nan,nan,0.100000
2,nan,nan,0.100000
3,nan,nan,0.100000
4,nan,nan,0.100000
5,nan,nan,0.100000
6,nan,nan,0.100000
7,nan,nan,0.100000
8,nan,nan,0.100000
9,nan,nan,0.100000
10,nan,nan,0.100000


Evaluating ArrowHead


epoch,train_loss,valid_loss,accuracy
1,1.087650,1.099914,0.302857
2,1.053009,1.117777,0.302857
3,1.052768,1.212124,0.297143
4,0.997668,1.380938,0.365714
5,0.958867,1.378320,0.400000
6,0.895489,0.880778,0.640000
7,0.845759,0.765900,0.628571
8,0.858950,0.732737,0.588571
9,0.826034,0.778130,0.554286
10,0.784061,0.810898,0.560000


epoch,train_loss,valid_loss,accuracy
1,0.580735,0.834260,0.537143
2,0.626181,0.872925,0.525714


In [17]:
rDF = pd.DataFrame(results,columns=["Name","Random","AE Loss","RandomCat","Concat"])

In [18]:
rDF["GainT"] = rDF["Concat"] - rDF["Random"]
rDF["GainC"] = rDF["Concat"] - rDF["RandomCat"]

In [20]:
rDF.to_csv("ucrresults.csv")

In [3]:
rDF = pd.read_csv("ucrresults.csv")

In [23]:
rDF.sort_values("GainT",ascending=False)

,Name,Random,AE Loss,RandomCat,Concat,GainT,GainC
69,Meat,0.333333,0.005748,0.950000,0.650000,0.316667,-0.300000
108,Strawberry,0.586486,0.010888,0.959459,0.832432,0.245946,-0.127027
5,ArrowHead,0.548571,0.100859,0.748571,0.788571,0.240000,0.040000
68,Mallat,0.776972,0.021429,0.939446,0.973134,0.196162,0.033689
73,MiddlePhalanxOutlineCorrect,0.570447,0.009673,0.800687,0.762887,0.192440,-0.037801
19,CricketZ,0.320513,0.484863,0.535897,0.502564,0.182051,-0.033333
92,ProximalPhalanxOutlineCorrect,0.646048,0.008137,0.890034,0.817869,0.171821,-0.072165
15,Coffee,0.857143,0.016528,1.000000,1.000000,0.142857,0.000000
37,FaceFour,0.647727,0.734557,0.840909,0.784091,0.136364,-0.056818
6,Beef,0.466667,0.167091,0.800000,0.600000,0.133333,-0.200000


In [1245]:
learnTransfer.freeze_to(1)

In [1247]:
learnTransfer.fit_one_cycle(10,1e-3,wd=0.2)

epoch,train_loss,valid_loss,accuracy
1,0.093543,0.272463,0.935111
2,0.082593,0.271288,0.935556
3,0.080639,0.269597,0.936000
4,0.080529,0.268113,0.936889
5,0.078305,0.268081,0.936222
6,0.080295,0.268183,0.936889
7,0.078981,0.268396,0.936667
8,0.079904,0.268938,0.936222
9,0.078470,0.270798,0.936222
10,0.076618,0.270624,0.936444


In [12]:
def conv1d(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias=False, init:LayerFunc=nn.init.kaiming_normal_) -> nn.Conv1d:
    "Create and initialize `nn.Conv1d` layer. `padding` defaults to `ks//2`."
    if padding is None: padding = ks//2
    return init_default(nn.Conv1d(ni, nf, kernel_size=ks, stride=stride, padding=padding, bias=bias), init)

def _bn1d(ni, init_zero=False):
    "Batchnorm layer with 0 initialization"
    m = nn.BatchNorm1d(ni)
    m.weight.data.fill_(0 if init_zero else 1)
    m.bias.data.zero_()
    return m

def bn_relu_conv1d(ni, nf, ks, stride, init_zero=False):
    bn_initzero = _bn1d(ni, init_zero=init_zero)
    return nn.Sequential(bn_initzero, nn.ReLU(inplace=True), conv1d(ni, nf, ks, stride))

class ResBlock(torch.nn.Module):
    def __init__(self, ni, nf, stride, drop_p=0.0):
        super().__init__()
        self.bn = nn.BatchNorm1d(ni)
        self.conv1 = conv1d(ni, nf, 3, stride)
        self.conv2 = bn_relu_conv1d(nf, nf, 3, 1)
        self.drop = nn.Dropout(drop_p, inplace=True) if drop_p else None
        self.shortcut = conv1d(ni, nf, 1, stride) if ni != nf else noop

    def forward(self, x):
        x2 = F.relu(self.bn(x), inplace=True)
        r = self.shortcut(x2)
        x = self.conv1(x2)
        if self.drop: x = self.drop(x)
        x = self.conv2(x) * 0.2
        return x.add_(r)

def _make_group(N, ni, nf, block, stride, drop_p):
    return [block(ni if i == 0 else nf, nf, stride if i == 0 else 1, drop_p) for i in range(N)]

class WideResNet1d(nn.Module):
    "Wide ResNet with `num_groups` and a width of `k`."
    def __init__(self, num_groups:int, N:int, k:int=1, drop_p:float=0.0, start_nf:int=16,maxY=20.0):
        super().__init__()
        self.maxY = maxY
        n_channels = [start_nf]
        for i in range(num_groups): n_channels.append(start_nf*(2**i)*k)
        
        
        layers = [conv1d(1, n_channels[0], 3, 1)]  # conv1
        self.split_groups = [layers[-1]] 
        
        for i in range(num_groups):
            layers += _make_group(N, n_channels[i], n_channels[i+1], ResBlock, (1 if i==0 else 2), drop_p)
            self.split_groups.append(layers[-N])
            
        layers += [nn.BatchNorm1d(n_channels[-1]), nn.ReLU(inplace=True), nn.AdaptiveAvgPool1d(1),
                   Flatten()]
        #self.split_groups.append(layers[-1])
        self.nf = n_channels[-1]
        self.features = nn.Sequential(*layers)

    def forward(self, x):
        #x = x.unsqueeze(1)
        return self.features(x)